In [3]:
import numpy as np
import os
import pickle

In [5]:
def getDataFromFile(filename):
    with open(filename, 'rb') as f:
        x = pickle.load(f)
        print(f'Retreived Data from {filename}')
        return x
def writeDataToFile(filename, data):
    outfile = open(filename,'wb')
    pickle.dump(dogs_dict,outfile)
    

In [7]:
filename = '/n/fs/scratch/djubas/largeRtts1b.pickle'
x = getDataFromFile(filename)

In [8]:
print(len(x))

59355
